# Bending algorithms

Bends is photonic-waveguides are very common and their efficiency can be crucial in highly optimized designs. In IPKISS there are several bending algoritms available and you can also make your own. Learning how to do this is the purpose of this notebook.

## 0. Setup

Let's first setup the technology and some rendering settings for the notebook.

In [ ]:
from technologies.silicon_photonics import TECH
from ipkiss3 import all as i3

In [ ]:
# Fix paths so that relative imports work in ipython 
import os, sys
sys.path.append('.')
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 6)

## Roundedwaveguides

Each waveguide with bends is created by the class i3.RoundedWaveguide which implements a rounding algorithm. To see what a rounding algorihm does it is best to start with an unrounded waveguide.

In [ ]:
shape = [(0.,0.),(10.,0.),(10.,10.),(20.,10.),(30.,20.)]
ur_wav = i3.Waveguide()
ur_wav_l = ur_wav.Layout(shape=shape, draw_control_shape=True)
ur_wav_l.visualize()

You see that a waveguide is drawn (using the default wg_template) and sharp corners (with no rounding) are used. The waveguide is built up by drawing **layers of a certain width** around a **center line**. **When no rounding algoritms are used the center line that is used is equal to the control shape** that is set as a layout parameter. To add bends the center line that is used needs to be rounded using that control shape as a starting point. Lets see how this is done using the a RoundedWaveguide.

In [ ]:
r_wav = i3.RoundedWaveguide()
r_wav_l = r_wav.Layout(shape=shape, bend_radius=1.0, draw_control_shape=True)
r_wav_l.visualize()

Now you see that the center line used in the waveguide is different from the control_shape and rounded. That center line can be extracted from the LayoutView of the waveguide.

In [ ]:
cl = r_wav_l.center_line_shape
plt.plot(cl.x_coords(),cl.y_coords())

A **rouding algorithm** is therefore a **shape_modifier** that yields a new shape for the center line given **the original control shape**. 

## Rounding Algoritms in IPKISS

The shape_modifiers used in waveguides IPKISS are:

- ShapeRound (by default)
- SplineRoundingAlgorithm


Lets have a look at how to use them.

In [ ]:
## ShapeRound
from ipkiss.geometry.shapes.modifiers import ShapeRound
rounded_shape = ShapeRound(original_shape=shape,radius=5.0)
plt.plot(rounded_shape.x_coords(),rounded_shape.y_coords())

# SplineRouding
from ipkiss.geometry.shapes.spline import ShapeRoundAdiabaticSpline
rounded_shape_spine = ShapeRoundAdiabaticSpline(original_shape=shape,radius=2.0,adiabatic_angles=(20,20))
plt.plot(rounded_shape_spine.x_coords(),rounded_shape_spine.y_coords())

If you want to pass your rounding algorithm to a waveguide class you can do it like this.

In [ ]:
my_rounding_alg=ShapeRound
r_wav = i3.RoundedWaveguide()
r_wav_l = r_wav.Layout(shape=shape, rounding_algorithm=my_rounding_alg, draw_control_shape=True)
r_wav_l.visualize()

my_rounding_alg=ShapeRoundAdiabaticSpline
r_wav = i3.RoundedWaveguide()
r_wav_l = r_wav.Layout(shape=shape, rounding_algorithm=my_rounding_alg, draw_control_shape=True)
r_wav_l.visualize()

## Creating your own rounding algoritms.

If you want to create your own algoritm best is to inherit from __ShapeModifierAutoOpenClosed__ and add the following required properties 
- angle_step and radius need to be there as properties. 
- the validation methods still assume that there is a bend_radius although that may be irrelevant for the rounding algorithm you are implementing.

If you only use your rounding algorithm in combination with i3.RoundedWaveguide it is sufficient that the original shape takes only 3 points. The start point, the bend_point and the end_point. 


Below is a rahter inefficient implementation of the ShapeRound algorithm. 

<img src="_images/shape_round.png" width=600>

It basically uses the property **original_shape** that is assumed to have 3 points, a start point, a bend points and an end point. Those points are transformed to a new shape that includes the start point and end point but adds a nice arc in the middle. Most of the logic in the class is math that calculates the center and the start and end angles of the arc.

In the method define_points, the complete set of points is returned, which again includes that start and end_point. I have also added a visualize() method as a help to understand how the algorithm works.

In [ ]:
from ipkiss.geometry.shape_modifier import __ShapeModifierAutoOpenClosed__
import numpy as np
norm = np.linalg.norm

def get_angle(p0, p1=np.array([0,0]), p2=None):
    ''' compute angle (in degrees) for p0p1p2 corner
    Inputs:
        p0,p1,p2 - points in the form of [x,y]
    '''
    if p2 is None:
        p2 = p1 + np.array([1, 0])
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return np.degrees(angle)

class MyRoundingAlgorithm(__ShapeModifierAutoOpenClosed__):
    angle_step = i3.PositiveNumberProperty(default=1.0, doc="Property required by RoundedWaveguide. The value can ignored if you do not need it")
    radius=i3.PositiveNumberProperty(default=1.0, doc="Property required by RoundedWaveguide. The value can ignored if you do not need it")
    
    def validate_properties(self):
        if len(self.original_shape) != 3.0:
            raise Exception("The length of the shape is not 3:{}".format(self.original_shape))
            
        return True
    
    ## This method returns the points of the new shape
    def define_points(self,pts):
        original_shape= self.original_shape
        new_points = [original_shape[0]]
        new_points.extend(self._get_arc_points())
        new_points.extend([original_shape[2]])
        return np.array(new_points)
    
    @i3.cache()
    def _get_angles(self):
        intersects = self._get_intersects()
        center = self._get_center_circle()
        angles = []
        angles.append(get_angle(center+(self.radius,0),center,intersects[0]))
        angles.append(get_angle(center+(self.radius,0),center,intersects[1])) 
        return (np.array(angles) + 360) % 360          
    
    @i3.cache()
    def _get_center_circle(self):
        """
        gets the center point of the arc 
        """
        original_shape = self.original_shape
        start_point = np.array(self.original_shape[0] - self.original_shape[1])
        end_point = np.array(self.original_shape[2] - self.original_shape[1])
        base_point = (start_point/norm(start_point) + end_point/norm(end_point))/2.0
        angle = get_angle(self.original_shape[0],self.original_shape[1],self.original_shape[2])/2.0
        sin_angle = np.sin(angle*np.pi/180.0)

        d = self.radius/sin_angle
        factor = d/np.linalg.norm(base_point)
        center = abs(factor) * base_point + self.original_shape[1]
        return center
    
    @i3.cache()
    def _get_intersects(self):
        c = self._get_center_circle()
        d = (norm(c-self.original_shape[1])**2 - self.radius**2)**0.5
        intersects = []
        for cnt in [0,2]:
            factor = d/norm(self.original_shape[cnt]-self.original_shape[1])
            intersects.append(abs(factor) * (self.original_shape[cnt]-self.original_shape[1]) + self.original_shape[1])
        return intersects   
    
    @i3.cache()
    def _get_arc_points(self):
        center = self._get_center_circle()
        sa = start_angle=self._get_angles()[0]
        ea = self._get_angles()[1]
        arc_left = i3.ShapeArc(center=center,start_angle=sa,end_angle=ea,radius=self.radius,clockwise=True,angle_step=self.angle_step)
        arc_right = i3.ShapeArc(center=center,start_angle=sa,end_angle=ea,radius=self.radius,clockwise=False,angle_step=self.angle_step)
        
        if arc_left.length() > arc_right.length():
            return arc_right
        else:
            return arc_left
        
    @i3.cache()    
    def _get_full_circle(self):
        center = self._get_center_circle()
        arc_points = i3.ShapeArc(center=center,start_angle=0.0,end_angle=360.0,radius=self.radius)
        return arc_points      
    
    def visualize(self):
        plt.plot(self.x_coords(),self.y_coords())
        plt.plot([p[0] for p in self.original_shape], [p[1] for p in self.original_shape], 'c')
        plt.plot(self._get_center_circle()[0],self._get_center_circle()[1],'*')
        plt.plot(self.points[:,0],self.points[:,1],'-*')
        plt.plot(self._get_full_circle().x_coords(),self._get_full_circle().y_coords(),'--')
        plt.plot(self._get_intersects()[0][0],self._get_intersects()[0][1],'*')
        plt.plot(self._get_intersects()[1][0],self._get_intersects()[1][1],'*')
    


original_shape = [(10.,0.),(10.,10.),(-10.,20.)]
rounded_shape = MyRoundingAlgorithm(original_shape=original_shape,radius=2)
rounded_shape.visualize()

The rounding algorithm can then be used in combination with RoundedWaveguide. 

In [ ]:
wav = i3.RoundedWaveguide()
wav_layout = wav.Layout(shape = shape, rounding_algorithm = MyRoundingAlgorithm , bend_radius=2.0)
wav_layout.visualize()